In [1]:
i=3

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import csv
import pickle
import seaborn as sns
import numpy as np
from scipy.optimize import curve_fit
from tqdm import tqdm
import matplotlib.pyplot as plt

import sys
sys.path.append("/home/mgander/Atlantic/notebooks/Kinobeads")
import fitter
from fitter import process_dfs
cols = ['-pec50', 'slope', 'front', 'back', 'fold_change', 'auc', 'rmse', 'r2', 'M1_noise', 'p_err_0', 'p_err_1', 'p_err_2', 'p_err_3', 
     'p_err_4', 'm0_rmse', 'm1_likelihood', 'm0_likelihood', 'f_statistic', 'p_value']

Path='/home/mgander/Atlantic/data/Viability/'

M = pd.read_pickle(f'{Path}/M_all_all.pkl')
sources = sorted(set(M['Source']))

df = M[M['Source']==sources[i]].copy()

df['log_dose']=np.log10(df['Dose'])

df['key'] = df['Cello']+'_'+df['PubChem_CID']
df['key'] = df['key'].astype('category')

keys=sorted(set(df['key']))

def do_batch(batch):
    dfb = df[df['key'].isin(batch)].copy()
    dfb['key'] = list(dfb['key'])
    dfb['key'] = dfb['key'].astype('category')
    all_dfs = [group for _, group in dfb.groupby('key')]

    from joblib import Parallel, delayed
    from tqdm import tqdm
    
    # Initialize lists to store results
    lks = []
    ress = []
    
    # Define the number of jobs
    num_jobs = 25  # Set to -1 to use all available cores
    
    # Parallel processing
    results = Parallel(n_jobs=num_jobs)(delayed(process_dfs)(dfs) for dfs in all_dfs)
    
    # Process the results
    for key, res in results:
        lks.append(key)
        ress.append(res)
    return(lks, ress)


lks, ress = [], []

batchsize = 10**4

ks = int(np.ceil(len(keys) / batchsize))

for k in tqdm(range(ks)):
    batch = keys[k*batchsize:(k+1)*batchsize]
    a,b = do_batch(batch)
    lks.append(a)
    ress.append(b)
lks2=[a for b in lks for a in b]
ress2=[a for b in ress for a in b]

dff=pd.DataFrame(data=np.array(ress2), index=lks2, columns=cols)
dff.to_pickle(f'{Path}/dff_gdsc1.pkl')
dff

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [19:38<00:00, 36.84s/it]


,-pec50,slope,front,back,fold_change,auc,rmse,r2,M1_noise,p_err_0,p_err_1,p_err_2,p_err_3,p_err_4,m0_rmse,m1_likelihood,m0_likelihood,f_statistic,p_value
22RV1_10027278,-0.716395,3.965240,0.993727,1.267328e-02,-6.292988,0.352525,0.923741,NaN,0.048883,1.258485,0.030449,0.063915,0.738767,0.401454,0.401454,NaN,NaN,99.986134,1.114186e-06
22RV1_10074640,-1.316712,20.769275,0.991910,8.845726e-05,-13.452941,0.381423,0.981416,NaN,198.342360,262687.658874,0.015095,0.045284,0.882743,0.282625,0.282625,NaN,NaN,172.286033,1.323788e-07
22RV1_10077147,-1.012544,0.983144,1.034643,2.940780e-03,-8.458715,0.404267,0.799507,NaN,0.477782,0.922035,0.068649,0.148759,0.883146,0.308554,0.308554,NaN,NaN,16.356766,9.178552e-04
22RV1_10109823,-1.695217,0.547912,1.006773,3.772549e-04,-11.276818,0.430591,0.815961,NaN,1.072770,0.580948,0.086519,0.139440,0.845035,0.290471,0.290471,NaN,NaN,16.530929,8.853094e-04
22RV1_10113978,-1.695520,1.220471,0.963435,7.289503e-05,-13.690078,0.411891,0.956044,NaN,1.452356,2.099014,0.028300,0.069101,0.869039,0.269318,0.269318,NaN,NaN,65.355232,5.761210e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZR7530_TRAIL,0.629703,3.011550,1.058805,3.149023e-07,-21.681029,0.372882,0.985637,NaN,4.471082,36.498735,0.025822,0.057378,0.916867,0.349778,0.349778,NaN,NaN,146.647996,1.520057e-04
ZR7530_TTK_3146,-4.296524,1.114713,1.011014,-7.914517e-07,NaN,0.610805,0.991206,NaN,2258.186399,765.071763,0.022193,0.044655,0.884603,0.335800,0.335800,NaN,NaN,229.934743,6.226750e-05
ZR7530_XMD15-27,-3.910250,0.403099,1.015713,-3.546495e-03,-15.788488,0.584374,0.993515,NaN,2.672647,0.361562,0.018542,0.038370,0.910191,0.275975,0.275975,NaN,NaN,461.498022,2.677538e-09
ZR7530_ZL049,-2.020293,0.819530,0.992619,3.747532e-05,-14.682015,0.442898,0.991465,NaN,0.967906,0.749323,0.023520,0.042450,0.844581,0.323124,0.323124,NaN,NaN,229.757312,6.236311e-05


In [2]:
(~np.isfinite(dff)).sum()

-pec50            35176
slope                 0
front                 0
back                  0
fold_change       90826
auc               35232
rmse                  0
r2               313321
M1_noise          35176
p_err_0           35176
p_err_1           35176
p_err_2           35176
p_err_3               0
p_err_4               0
m0_rmse               0
m1_likelihood    313321
m0_likelihood    313321
f_statistic           0
p_value               0
dtype: int64

In [2]:
i=7

import pandas as pd
import numpy as np
import csv
import pickle
import seaborn as sns
import numpy as np
from scipy.optimize import curve_fit
from tqdm import tqdm
import matplotlib.pyplot as plt

import sys
sys.path.append("/home/mgander/Atlantic/notebooks/Kinobeads")
import fitter
from fitter import process_dfs

Path='/home/mgander/Atlantic/data/Viability/'
dff = pd.read_pickle(f'{Path}/dff_gdsc1.pkl')

In [3]:
(~np.isfinite(dff)).sum()

-pec50            35176
slope                 0
front                 0
back                  0
fold_change       90826
auc               35232
rmse                  0
r2               313321
M1_noise          35176
p_err_0           35176
p_err_1           35176
p_err_2           35176
p_err_3               0
p_err_4               0
m0_rmse               0
m1_likelihood    313321
m0_likelihood    313321
f_statistic           0
p_value               0
dtype: int64

In [ ]:
i=3

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import csv
import pickle
import seaborn as sns
import numpy as np
from scipy.optimize import curve_fit
from tqdm import tqdm
import matplotlib.pyplot as plt

import sys
sys.path.append("/home/mgander/Atlantic/notebooks/Kinobeads")
import fitter_exhaustive
from fitter_exhaustive import process_dfs
cols = ['-pec50', 'slope', 'front', 'back', 'fold_change', 'auc', 'rmse', 'r2', 'M1_noise', 'p_err_0', 'p_err_1', 'p_err_2', 'p_err_3', 
     'p_err_4', 'm0_rmse', 'm1_likelihood', 'm0_likelihood', 'f_statistic', 'p_value']

Path='/home/mgander/Atlantic/data/Viability/'

M = pd.read_pickle(f'{Path}/M_all_all.pkl')
sources = sorted(set(M['Source']))

df = M[M['Source']==sources[i]].copy()
df['log_dose']=np.log10(df['Dose'])

df['key'] = df['Cello']+'_'+df['PubChem_CID']
df['key'] = df['key'].astype('category')
df = df[df['key'].isin(list(dff[~np.isfinite(dff['-pec50'])].index))].copy()

keys=sorted(set(df['key']))

def do_batch(batch, k):
    pd.DataFrame({'k':[k]}).to_csv('tracker.csv')
    dfb = df[df['key'].isin(batch)].copy()
    dfb['key'] = list(dfb['key'])
    dfb['key'] = dfb['key'].astype('category')
    all_dfs = [group for _, group in dfb.groupby('key')]

    from joblib import Parallel, delayed
    from tqdm import tqdm
    
    # Initialize lists to store results
    lks = []
    ress = []
    
    # Define the number of jobs
    num_jobs = 28  # Set to -1 to use all available cores
    
    # Parallel processing
    results = Parallel(n_jobs=num_jobs)(delayed(process_dfs)(dfs) for dfs in all_dfs)
    
    # Process the results
    for key, res in results:
        lks.append(key)
        ress.append(res)
    return(lks, ress)


lks, ress = [], []

batchsize = 10**3

ks = int(np.ceil(len(keys) / batchsize))
'
for k in tqdm(range(ks)):
    batch = keys[k*batchsize:(k+1)*batchsize]
    a,b = do_batch(batch, k)
    lks.append(a)
    ress.append(b)
lks2=[a for b in lks for a in b]
ress2=[a for b in ress for a in b]

dff2=pd.DataFrame(data=np.array(ress2), index=lks2, columns=cols)

 50%|█████████████████████████████████████████▌                                         | 18/36 [5:48:33<5:53:41, 1178.97s/it]

In [5]:
(~np.isfinite(dff2)).sum()

-pec50             314
slope                0
front                0
back                 0
fold_change       4911
auc               1038
rmse                 0
r2                   0
M1_noise           314
p_err_0            314
p_err_1            314
p_err_2            314
p_err_3              0
p_err_4              0
m0_rmse              0
m1_likelihood        0
m0_likelihood    35176
f_statistic          0
p_value              0
dtype: int64

In [6]:
dffnn =  dff[np.isfinite(dff['-pec50'])].copy()
dff666 = pd.concat([dffnn, dff2])
dff666.to_pickle(f'{Path}/dff_gdsc1.pkl')